In [1]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *

## Getting set up

In [2]:
comp = 'playground-series-s3e25'
path = setup_comp(comp, install='')

In [3]:
path

Path('playground-series-s3e25')

In [4]:
trn_path = path/'train.csv'

In [5]:
import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, FunctionTransformer, OneHotEncoder,QuantileTransformer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, median_absolute_error as medae
from scipy.stats import loguniform

In [6]:
df = pd.read_csv(trn_path)
X = df.iloc[:,1:-1] # drop id and target
y = df.iloc[:,-1]
X_tr, X_dev, y_tr, y_dev = train_test_split(X,y,test_size=0.2) # XXyy

In [7]:
nums = [1.75, 2.55, 3.75, 4.75,5.75,6.55,7.75,8.75,9.75]
def quantize(y):
    return y.apply(lambda o: nums[np.abs(np.array(nums)-o).argmin()]).astype(str)

model = HistGradientBoostingClassifier()
y_pred = model.fit(X_tr, quantize(y_tr)).predict(X_dev).astype(float)
medae(y_pred, y_dev)

0.25

## Submitting to Kaggle

In [8]:
ss = pd.read_csv(path/'sample_submission.csv')

In [9]:
tst = pd.read_csv(path/'test.csv')

In [10]:
ss['Hardness'] = model.predict(tst.iloc[:,1:])

In [11]:
ss.to_csv('subm.csv', index=False)
!head subm.csv

id,Hardness
10407,2.55
10408,2.55
10409,5.75
10410,2.55
10411,5.75
10412,5.75
10413,3.75
10414,2.55
10415,1.75


In [16]:
!kaggle competitions submit -c 'playground-series-s3e25' -f subm.csv -m 'quantisation'

100%|███████████████████████████████████████| 74.6k/74.6k [00:00<00:00, 110kB/s]
Successfully submitted to Regression with a Mohs Hardness Dataset

In [ ]:
if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('subm.csv', 'quantization', comp)

## Conclusion

## Addendum

In [ ]:
if not iskaggle:
    push_notebook('xy', 'mock-first-steps-road-to-the-top-part-1',
                  title='mock First Steps: Road to the Top, Part 1',
                  file='mock-first-steps-road-to-the-top-part-1.ipynb',
                  competition=comp, private=False, gpu=True)